In [1]:
!pip install -q langchain-openai langchain-core requests

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import requests

d:\Generative AI\src\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [ ]:
# tool binding

In [10]:
llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash', temperature = 0.5)

In [11]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7abd6ae8-9954-419b-9d85-5aa7d89ffdad-0', usage_metadata={'input_tokens': 2, 'output_tokens': 43, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 33}})

In [12]:
llm_with_tools = llm.bind_tools([multiply])

In [13]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--55ef503e-41de-445c-a639-1bdc024163cc-0', usage_metadata={'input_tokens': 58, 'output_tokens': 16, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [20]:
query = HumanMessage('can you multiply 5 with 1000')

In [21]:
messages = [query]

In [22]:
messages

[HumanMessage(content='can you multiply 5 with 1000', additional_kwargs={}, response_metadata={})]

In [23]:
result = llm_with_tools.invoke(messages)

In [24]:
messages.append(result)

In [25]:
messages

[HumanMessage(content='can you multiply 5 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 5, "b": 1000}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ba939484-f2ae-4f18-877d-1b47e854cbd1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 1000}, 'id': '7e04b9cb-4079-44a6-8f48-5e7ddc57e9b9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 94, 'total_tokens': 159, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 73}})]

In [26]:
tool_result = multiply.invoke(result.tool_calls[0])

In [27]:
tool_result

ToolMessage(content='5000', name='multiply', tool_call_id='7e04b9cb-4079-44a6-8f48-5e7ddc57e9b9')

In [28]:
messages.append(tool_result)

In [30]:
messages

[HumanMessage(content='can you multiply 5 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 5, "b": 1000}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ba939484-f2ae-4f18-877d-1b47e854cbd1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 1000}, 'id': '7e04b9cb-4079-44a6-8f48-5e7ddc57e9b9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 94, 'total_tokens': 159, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 73}}),
 ToolMessage(content='5000', name='multiply', tool_call_id='7e04b9cb-4079-44a6-8f48-5e7ddc57e9b9')]

In [31]:
llm_with_tools.invoke(messages).content

'The product of 5 and 1000 is 5000.'

# **Currency Conversion Tool**

In [32]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [33]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [34]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1763251202,
 'time_last_update_utc': 'Sun, 16 Nov 2025 00:00:02 +0000',
 'time_next_update_unix': 1763337602,
 'time_next_update_utc': 'Mon, 17 Nov 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.7636}

In [35]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [36]:
# tool binding
llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash', temperature=0.2)

In [37]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [38]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [39]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [40]:
ai_message = llm_with_tools.invoke(messages)

In [41]:
messages.append(ai_message)

In [43]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': 'f833b42b-caf4-416f-89a9-827b2f21c654',
  'type': 'tool_call'}]

In [44]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [45]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--667fdc4b-a365-4305-abcd-d0e4d1d39ff4-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': 'f833b42b-caf4-416f-89a9-827b2f21c654', 'type': 'tool_call'}], usage_metadata={'input_tokens': 150, 'output_tokens': 525, 'total_tokens': 675, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 499}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_u

In [46]:
llm_with_tools.invoke(messages).content

''

In [47]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\baibh\AppData\Local\Temp\ipykernel_19664\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [48]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: I am an AI and do not have feelings. I should respond in a helpful and polite manner.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Hello! I'm functioning well and ready to assist you. How can I help you today?"
}
```

> Finished chain.
